In [1]:
import pandas as pd 
import numpy as np 
import os 
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score,roc_auc_score,mean_squared_error
import warnings
warnings.filterwarnings(action='ignore')
import random
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

In [2]:
train_data=pd.read_csv('/home/iai/son/lastcheck/spambase_train_data.csv')
test_data=pd.read_csv('/home/iai/son/lastcheck/spambase_test_data.csv')

In [3]:
train_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20','0.96']
test_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20']

In [4]:
train_X=train_data.loc[:,train_stage_features]
train_y=train_data['1']
new_test_data_X=test_data.loc[:,test_stage_features]
new_test_data_y=test_data['1']

In [5]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [6]:
# train에는 있는데 test data에는 없는 컬럼 찾기

train_data_columns=np.array(train_X.columns)
test_data_columns=np.array(new_test_data_X.columns)
np.setdiff1d(train_data_columns,test_data_columns)

array(['0.96'], dtype=object)

In [7]:

import optuna
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

test_data_columns=test_stage_features
y='0.96'

train_X_new=train_X.loc[:,test_data_columns]
train_y_new=train_X.loc[:,y]



# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X_new, train_y_new,
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=10)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)



[I 2023-04-26 15:41:53,558] A new study created in memory with name: no-name-0eed969e-135c-49d0-aa6b-034c6eacc550
[I 2023-04-26 15:41:54,787] Trial 0 finished with value: 0.8560944942676644 and parameters: {'n_estimators': 443, 'max_depth': 7, 'learning_rate': 0.02199825365749966}. Best is trial 0 with value: 0.8560944942676644.
[I 2023-04-26 15:41:55,858] Trial 1 finished with value: 0.8505050451886884 and parameters: {'n_estimators': 884, 'max_depth': 5, 'learning_rate': 0.04202845262638555}. Best is trial 1 with value: 0.8505050451886884.
[I 2023-04-26 15:41:57,147] Trial 2 finished with value: 0.8530636212081486 and parameters: {'n_estimators': 649, 'max_depth': 8, 'learning_rate': 0.043106832401420714}. Best is trial 1 with value: 0.8505050451886884.
[I 2023-04-26 15:41:57,816] Trial 3 finished with value: 0.8553789746871852 and parameters: {'n_estimators': 382, 'max_depth': 7, 'learning_rate': 0.02967309335767157}. Best is trial 1 with value: 0.8505050451886884.
[I 2023-04-26 15:

{'n_estimators': 884, 'max_depth': 5, 'learning_rate': 0.04202845262638555}


In [8]:
# Print best hyperparameters and auc

print(f'Best hyperparameters: {study.best_params}')
print(f'Best RMSE: {study.best_value:.4f}')

Best hyperparameters: {'n_estimators': 884, 'max_depth': 5, 'learning_rate': 0.04202845262638555}
Best RMSE: 0.8505


In [9]:
(est,depth,rate)=study.best_params.values()

In [10]:
def bestreg_parametertuning(rate,depth,est,test_data_columns,y):
    best_lgbmreg=LGBMRegressor(learning_rate=rate,max_depth=depth,n_estimators=est,random_state=42)
    best_lgbmreg.fit(train_X[test_data_columns], train_X[y])
    new_test_data_X[y]=best_lgbmreg.predict(new_test_data_X[test_data_columns])
    

In [11]:
bestreg_parametertuning(rate,depth,est,test_data_columns,'0.96')

In [12]:
new_test_data_X

,0.7,278,61,0,0.34,0.30,0.5,0.44,0.40,0.778,0.24,1.93,0.39,1.29,0.43,0.2,0.36,0.17,0.20,0.96
0,0.00,3,1,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,-0.014747
1,0.00,32,2,0.71,0.0,0.0,0.00,0.000,0.0,0.000,0.0,1.43,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.607997
2,0.00,133,51,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,1.09,0.0,0.143300
3,1.26,290,287,0.00,0.0,0.0,1.26,0.000,0.0,0.537,0.0,2.53,0.0,0.00,0.0,0.00,0.0,0.00,0.0,3.769185
4,0.00,23,3,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,3.75,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.668797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,0.14,898,217,0.14,0.0,0.0,1.47,0.215,0.0,1.004,0.0,2.35,0.0,1.62,0.0,0.29,0.0,0.00,0.0,2.056721
916,0.00,740,54,0.09,0.0,0.0,0.09,0.979,0.0,5.213,0.0,3.75,0.0,0.00,0.0,0.19,0.0,0.00,0.0,1.296863
917,0.00,3,1,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,-0.014747
918,0.00,92,11,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,1.12,0.0,0.00,0.0,0.00,0.0,0.56,0.0,0.460694


In [13]:
import lightgbm as lgb

In [14]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=10)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-26 15:42:34,424] A new study created in memory with name: lgb_boost_opt
[I 2023-04-26 15:42:36,604] Trial 0 finished with value: 0.9682281901794564 and parameters: {'learning_rate': 0.012976352518534484, 'max_depth': 7, 'n_estimators': 820}. Best is trial 0 with value: 0.9682281901794564.
[I 2023-04-26 15:42:38,354] Trial 1 finished with value: 0.9683193738631031 and parameters: {'learning_rate': 0.017427210583795393, 'max_depth': 6, 'n_estimators': 709}. Best is trial 1 with value: 0.9683193738631031.
[I 2023-04-26 15:42:39,178] Trial 2 finished with value: 0.9649793114949535 and parameters: {'learning_rate': 0.019070090964182593, 'max_depth': 3, 'n_estimators': 905}. Best is trial 1 with value: 0.9683193738631031.
[I 2023-04-26 15:42:39,412] Trial 3 finished with value: 0.9672773032910726 and parameters: {'learning_rate': 0.09791837669113711, 'max_depth': 4, 'n_estimators': 171}. Best is trial 1 with value: 0.9683193738631031.
[I 2023-04-26 15:42:41,377] Trial 4 finished w

Best hyperparameters: {'learning_rate': 0.017427210583795393, 'max_depth': 6, 'n_estimators': 709}
Best AUC: 0.9683


In [15]:
lgb=lgb.LGBMClassifier(learning_rate= 0.017427210583795393,max_depth=6,n_estimators=709,scale_pos_weight=ratio,random_state=42)

In [16]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.017427210583795393, max_depth=6,
               n_estimators=709, random_state=42,
               scale_pos_weight=1.5879043600562588)

In [17]:
pred=lgb.predict_proba(new_test_data_X)[:,1]

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [19]:
get_clf_prob(new_test_data_y,pred) 

임계값:  0.5
오차행렬
[[491  39]
 [ 61 329]]
정확도: 0.8913, 정밀도: 0.8940, 재현율: 0.8436, F1:0.8878, AUC: 0.9520
